In [ ]:
import pandas as pd
from tqdm import tqdm
import numpy as np
from scipy import interpolate

import sys
ids_pd = pd.read_csv("../utils/intersection_ids.csv")
ids = ids_pd['nhdhr_id'].to_list()

base_path = "../data/raw/temperature/"
path_extensions = ["ice_flags","meteo","predictions","cfg", "obs"]


raw_path = '../data/raw/dissolved_oxygen/old_lake_data_all.csv'
raw_data = pd.read_csv(raw_path)
raw_data = raw_data.drop_duplicates(subset=['nhdhr_id', 'datetime'])

In [2]:
import os
def create_path(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [3]:
def ensure_naive(df, col_name):
    """Ensure the datetime column is timezone naive."""
    df[col_name] = pd.to_datetime(df[col_name]).dt.tz_localize(None)
    return df

def expand_with_depths(df, depth_values, date_col):
    """Expand daily data to include depth information."""
    df[date_col] = pd.to_datetime(df[date_col])
    df = df.set_index(date_col)
    all_dates = pd.date_range(start_date, end_date)
    df = df.reindex(all_dates).ffill().reset_index()
    df.rename(columns={'index': date_col}, inplace=True)
    expanded = []
    for depth in depth_values:
        temp_df = df.copy()
        temp_df['vertical_depth'] = depth
        expanded.append(temp_df)
    expanded_df = pd.concat(expanded).reset_index(drop=True)
    return expanded_df

In [4]:
def fill_do_features(feature_df):
# Delete first day of 1979
    # feature_df = feature_df.iloc[1:,:].copy()
    feature_df = feature_df
    # # Morphometric_list    
    # Morphometric_list = ['Shore_len', 'Vol_total', 'Vol_res', 'Vol_src','Depth_avg', 
    # 'Dis_avg', 'Res_time', 'Elevation', 'Slope_100','Wshd_area']
    # for feature in Morphometric_list:
    #     missing_count = feature_df[feature].isna().sum()
    #     if feature_df[feature].isna().sum() == len(feature_df[feature]):
    #         mean_value = mean_std.loc[mean_std['feature'] == feature, 'mean'].values[0]
    #         # std_value = mean_std.loc[mean_std['feature'] == feature, 'std'].values[0]
    #         # noise = np.random.normal(loc=mean_value, scale=std_value, size=missing_count)
    #         feature_df.loc[feature_df[feature].isna(), feature] = mean_value
    #         # To-DO 

    #     elif missing_count > 0:
    #         feature_df[feature].fillna(feature_df[feature].mean(), inplace=True)

    # Flux data
    flux1 = ['fnep', 'fmineral', 'fsed', 'fatm', 'fdiff']
    for flux in flux1:
        feature_df[flux].fillna(method='bfill', inplace=True)

    flux2 = ['fentr_epi', 'fentr_hyp']
    for flux in flux2:
        feature_df[flux].fillna(0, inplace=True)
    
    # #Land use
    # lanuse_list = ['water', 'developed',
    # 'barren', 'forest', 'shrubland', 'herbaceous', 'cultivated', 'wetlands']
    # for feature in lanuse_list:
    #     missing_count = feature_df[feature].isna().sum()
    #     if feature_df[feature].isna().sum() == len(feature_df[feature]):
    #         mean_value = mean_std.loc[mean_std['feature'] == feature, 'mean'].values[0]
    #         # std_value = mean_std.loc[mean_std['feature'] == feature, 'std'].values[0]
    #         # noise = np.random.normal(loc=mean_value, scale=std_value, size=missing_count)
    #         feature_df.loc[feature_df[feature].isna(), feature] = mean_value
    #         # To-DO 

    #     elif missing_count > 0:
    #         feature_df[feature].fillna(feature_df[feature].mean(), inplace=True)

    # Trophic State and Classification
    # noise_list = ['eutro', 'oligo', 'dys', 'sat_hypo', 'thermocline_depth']
    # for feature in noise_list:
    #     mean_value = feature_df[feature].mean()
    #     std_value = feature_df[feature].std()
    #     missing_count = feature_df[feature].isna().sum()

    #     if feature_df[feature].isna().sum() == len(feature_df[feature]):
    #         mean_value = mean_std.loc[mean_std['feature'] == feature, 'mean'].values[0]
    #         std_value = mean_std.loc[mean_std['feature'] == feature, 'std'].values[0]
    #         noise = np.random.normal(loc=mean_value, scale=std_value, size=missing_count)
    #         feature_df.loc[feature_df[feature].isna(), feature] = noise

    #     elif missing_count > 0:
    #         noise = np.random.normal(loc=mean_value, scale=std_value, size=missing_count)
    #         feature_df.loc[feature_df[feature].isna(), feature] = noise

    # temperature_list = ['temperature_epi', 'temperature_hypo']
    # # compare temperature_epi temperature_hypo nan
    # mask1 = feature_df[temperature_list[0]].isna()
    # mask2 = feature_df[temperature_list[1]].isna()
    # if not mask1.equals(mask2):
    #     raise Exception("temperature_epi and temperature_hypo not match")
    
    # assert not feature_df['temperature_total'].isna().any(), "temperature_total column contains NaN values"
    # # 用 temperature_total 替换 temperature_epi 和 temperature_hypo 中的 NaN 值
    # for feature in temperature_list:
    #     feature_df[feature] = feature_df[feature].fillna(feature_df['temperature_total'])

    # assert not feature_df['temperature_epi'].isna().any(), "temperature_epi column contains NaN values"

    valid_row = feature_df.dropna(subset=['volume_epi', 'volume_hypo']).iloc[0]

    volume_total = valid_row['volume_epi'] + valid_row['volume_hypo']

    
    feature_df['volume_epi'].fillna(volume_total, inplace=True)
    feature_df['volume_hypo'].fillna(0, inplace=True)


    # put obs_total in obs_epi
    overlap_condition = (~feature_df['obs_tot'].isna()) & (~feature_df['obs_epi'].isna())

    if overlap_condition.any():
        raise ValueError("Error: Overlap detected between 'obs_tot' and 'obs_epi'.")

    feature_df.loc[~feature_df['obs_tot'].isna(), 'obs_epi'] = feature_df['obs_tot']
 
    # when mixed, set sim_hypo = nan
    # feature_df.loc[mixed_condition, 'sim_hyp'] = np.nan

    # for feature in USE_FEATURES_COLUMNS:
    #     all_not_na = feature_df[feature].notna().all()
    #     if not all_not_na.all():
    #         raise Exception(f"Data missing in {feature}")
        
    return feature_df

In [ ]:
import pandas as pd
import numpy as np
from scipy import interpolate
from utils.utils import DROP_COLUMNS

# base_path, lake_id, dates_set, ids

start_date = pd.to_datetime('1980-01-01').tz_localize(None)
end_date = pd.to_datetime('2019-12-31').tz_localize(None)
raw_grouped = raw_data.groupby('nhdhr_id')


for lake_id, do_data_pd in tqdm(raw_grouped):
    do_data_pd = ensure_naive(do_data_pd, 'datetime')
    do_data_pd = do_data_pd.drop(columns=DROP_COLUMNS)
    # print("Filling do data...")
    do_data_pd = fill_do_features(do_data_pd)
    do_data_pd = do_data_pd[(do_data_pd['datetime'] >= start_date) & (do_data_pd['datetime'] <= end_date)]
    
    
    ### Ice_flag info
    ice_flags_df = pd.read_csv(base_path + 'ice_flags/pb0_' + lake_id + '_ice_flags.csv')
    ice_flags_df = ensure_naive(ice_flags_df, 'date')
    ice_flags_df = ice_flags_df[(ice_flags_df['date'] >= start_date) & (ice_flags_df['date'] <= end_date)]

    ### input features(ice_flag not included)
    meteo_df = pd.read_csv(base_path + 'meteo/' + lake_id + '_meteo.csv', delimiter=',', usecols=['time', 'ShortWave', 'LongWave', 'AirTemp', 'RelHum', 'WindSpeed', 'Rain', 'Snow'])
    meteo_df = ensure_naive(meteo_df, 'time')
    meteo_df = meteo_df[(meteo_df['time'] >= start_date) & (meteo_df['time'] <= end_date)]


    do_data_pd['datetime'] = pd.to_datetime(do_data_pd['datetime'])
    ice_flags_df['datetime'] = pd.to_datetime(ice_flags_df['date'])
    meteo_df['datetime'] = pd.to_datetime(meteo_df['time'])

    data_frames = [do_data_pd, ice_flags_df, meteo_df]


    row_counts = [df.shape[0] for df in data_frames]


    if len(set(row_counts)) != 1:
        raise Exception("DataFrames do not have the same number of rows")


    all_data = [df.set_index(['datetime']) for df in data_frames]

    merged_df = pd.concat(all_data, axis=1).reset_index()

    DROP_COLUMNS2 = ['date', 'time']
    merged_df = merged_df.drop(columns=DROP_COLUMNS2)




    # print(merged_df.head())
    # print("Result shape:", merged_df.shape)

    create_path("../data/raw/lakes/")
    merged_df.to_csv(f'../data/raw/lakes/merged_{lake_id}.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np
from scipy import interpolate
from utils.utils import DROP_COLUMNS

# base_path, lake_id, dates_set, ids

start_date = pd.to_datetime('1980-01-01').tz_localize(None)
end_date = pd.to_datetime('2019-12-31').tz_localize(None)
raw_grouped = raw_data.groupby('nhdhr_id')


for lake_id, do_data_pd in tqdm(raw_grouped):
    do_data_pd = ensure_naive(do_data_pd, 'datetime')
    do_data_pd = do_data_pd.drop(columns=DROP_COLUMNS)
    # print("Filling do data...")
    do_data_pd = fill_do_features(do_data_pd)
    do_data_pd = do_data_pd[(do_data_pd['datetime'] >= start_date) & (do_data_pd['datetime'] <= end_date)]
    
    
    ### Ice_flag info
    ice_flags_df = pd.read_csv(base_path + 'ice_flags/pb0_' + lake_id + '_ice_flags.csv')
    ice_flags_df = ensure_naive(ice_flags_df, 'date')
    ice_flags_df = ice_flags_df[(ice_flags_df['date'] >= start_date) & (ice_flags_df['date'] <= end_date)]

    ### input features(ice_flag not included)
    meteo_df = pd.read_csv(base_path + 'meteo/' + lake_id + '_meteo.csv', delimiter=',', usecols=['time', 'ShortWave', 'LongWave', 'AirTemp', 'RelHum', 'WindSpeed', 'Rain', 'Snow'])
    meteo_df = ensure_naive(meteo_df, 'time')
    meteo_df = meteo_df[(meteo_df['time'] >= start_date) & (meteo_df['time'] <= end_date)]

    # create temp_df (features, nhdhr_id, datetime, depth, glm_temp, obs_temp, 7 meteo('ShortWave', 'LongWave', 'AirTemp', 'RelHum', 'WindSpeed', 'Rain', 'Snow'))
    # print("starting: ", lake_id)

    # GLM
    glm_df = pd.read_csv(base_path + 'predictions/pb0_' + lake_id + '_temperatures.csv')
    glm_df = ensure_naive(glm_df, 'date')
    glm_df = glm_df[(glm_df['date'] >= start_date) & (glm_df['date'] <= end_date)]
    n_depths = glm_df.shape[1] - 1  # 减去日期列
    max_depth = 0.5 * (n_depths - 1)
    depths = np.arange(0, max_depth + 0.5, 0.5)

    # 初始化sim_save数组，形状为(n_depths, n_dates)
    sim_save = glm_df.iloc[:, 1:].values.T

 
    # print("Fill nan data in sim temperature...")
    if np.isnan(np.sum(sim_save)):
        for i in range(n_depths):
            for t in range(sim_save.shape[1]):
                if np.isnan(sim_save[i, t]):
                    x = depths[i]
                    xp = depths[0:(i)]
                    yp = sim_save[0:(i), t]
                    if xp.shape[0] == 1:
                        sim_save[i, t] = sim_save[i - 1, t]
                    else:
                        f = interpolate.interp1d(xp, yp, fill_value="extrapolate")
                        sim_save[i, t] = f(x)


    obs_df = pd.read_csv(base_path + 'obs/' + lake_id + '_obs.csv', usecols=['date', 'depth', 'temp'])
    obs_df = obs_df.rename(columns={'depth': 'vertical_depth'})
    obs_df = ensure_naive(obs_df, 'date')
    obs_df = obs_df[(obs_df['date'] >= start_date) & (obs_df['date'] <= end_date)]
    obs_df = obs_df[obs_df['vertical_depth'] <= max_depth]
    obs_df.sort_values(by='date', inplace=True)


    depth_indices = {depth: idx for idx, depth in enumerate(depths)}
    dates_indices = pd.Series(index=pd.to_datetime(glm_df.iloc[:, 0]), data=np.arange(len(glm_df)))


    # print("Processing glm temperature data...")
    temperature = []


    dates = pd.to_datetime(glm_df.iloc[:, 0])
    date_depth_pairs = [(d, depth) for d in dates for depth in depths]
    temperature_df = pd.DataFrame(date_depth_pairs, columns=['datetime', 'vertical_depth'])
    temperature_df['lake_id'] = lake_id
    temperature_df = pd.DataFrame(date_depth_pairs, columns=['datetime', 'vertical_depth'])

    reshaped_sim_save = sim_save.T.flatten()
    temperature_df['sim_temp'] = reshaped_sim_save



    # print("Processing obs temperature data...")
    temperature_df = temperature_df.merge(obs_df, left_on=['datetime', 'vertical_depth'], right_on=['date', 'vertical_depth'], how='left').drop(columns='date')
    temperature_df.rename(columns={'temp': 'obs_temp'}, inplace=True)


    # print("expand do data...")
    do_data_expanded = expand_with_depths(do_data_pd, depths, 'datetime')
    # print("expand ice_flag data...")
    ice_flags_expanded = expand_with_depths(ice_flags_df, depths, 'date')
    # print("expand meteo data...")
    meteo_expanded = expand_with_depths(meteo_df, depths, 'time')
    # print("do_data_expanded:", do_data_expanded)

    do_data_expanded['datetime'] = pd.to_datetime(do_data_expanded['datetime'])
    ice_flags_expanded['datetime'] = pd.to_datetime(ice_flags_expanded['date'])
    meteo_expanded['datetime'] = pd.to_datetime(meteo_expanded['time'])

    data_frames = [temperature_df, do_data_expanded, ice_flags_expanded, meteo_expanded]


    row_counts = [df.shape[0] for df in data_frames]


    if len(set(row_counts)) != 1:
        raise Exception("DataFrames do not have the same number of rows")


    all_data = [df.set_index(['datetime', 'vertical_depth']) for df in data_frames]

    merged_df = pd.concat(all_data, axis=1).reset_index()

    # Adding layer feature

    merged_df['layer'] = np.where(merged_df['vertical_depth'] < merged_df['thermocline_depth'], 0, 1)
    
    DROP_COLUMNS2 = ['date', 'time']
    merged_df = merged_df.drop(columns=DROP_COLUMNS2)

    # print(merged_df.head())
    # print("Result shape:", merged_df.shape)

    merged_df.to_csv(f'../data/raw/lakes/merged_{lake_id}.csv', index=False)


In [ ]:
import pandas as pd
import json
import os
import numpy as np
import re
def create_path(path):
    if not os.path.exists(path):
        os.makedirs(path)
    return path
base_path = "../data/raw/temperature/"
cfg_path = base_path+"cfg/pb0_config.json"
obs_df = pd.read_csv(base_path+"obs/temperature_observations.csv")
ids = np.unique(obs_df['site_id'].values)

cfg_f = open(cfg_path)
cfg = json.load(cfg_f)

ct = 0

for nid in ids: #for each new additional lake
    ct += 1
    print("processing lake ",ct,": ",nid)
    nid = str(nid)
    csv = []
    csv.append("depths,areas")
    m = re.search('nhdhr_(.*)', nid) 
    name = m.group(1)

    hs = np.array(cfg[nid]['morphometry']['H'])
    As = cfg[nid]['morphometry']['A']

    #get into posi depth form
    hs = -(np.flip(hs,axis=0) - hs.max())
    As = np.flip(As,axis=0)
    assert hs.shape[0] == As.shape[0]

    #back to str form
    hs = [str(i) for i in hs]
    As = [str(i) for i in As]

    for i in range(len(hs)):
        csv.append(",".join([hs[i], As[i]]))

    
    create_path(f"processedByLake/{nid}/")


    with open("processedByLake/"+nid+"/geometry",'w') as file:
        for line in csv:
            file.write(line)
            file.write('\n')


In [ ]:
import pandas as pd
import os
for dataset in ['train', 'test', 'val']:
    bucket_do_pd = pd.read_csv(f"processed/processed_{dataset}_do_processed.0.csv")
    bucket_temp_pd = pd.read_csv(f"processed/processed_{dataset}_temp_processed.0.csv")

    for lake_id, group in bucket_do_pd.groupby('nhdhr_id'):
        output_dir = f"processedByLake/{lake_id}"
        os.makedirs(output_dir, exist_ok=True)

        output_file = f"{output_dir}/bucket_do_{lake_id}_{dataset}.csv"
        group.to_csv(output_file, index=False)

        print(f"Saved {output_file}")

    for lake_id, group in bucket_temp_pd.groupby('nhdhr_id'):
        output_dir = f"processedByLake/{lake_id}"
        os.makedirs(output_dir, exist_ok=True)

        output_file = f"{output_dir}/bucket_temp_{lake_id}_{dataset}.csv"
        group.to_csv(output_file, index=False)

        print(f"Saved {output_file}")